In [ ]:
from pyspark import SparkContext

In [ ]:
sc = SparkContext(master="local",appName="transformacionesYAcciones")

In [ ]:
rddl = sc.parallelize([1,2,3])
type(rddl)

In [ ]:
rddl.collect()

In [ ]:
!ls curso-apache-spark-platzi/

In [ ]:
path = "curso-apache-spark-platzi/files/"

equiposOlimpicosRDD = sc.textFile(path + "paises.csv") \
                        .map(lambda line : line.split(","))

In [ ]:
equiposOlimpicosRDD.take(15)

In [ ]:
#sc.stop()

### ACCIONES DE MODIFICACIÓN SOBRE RDDs

In [ ]:
equiposOlimpicosRDD.map(lambda x : x[2]).distinct().count()

In [ ]:
equiposOlimpicosRDD.map(lambda x : (x[2],x[1])).groupByKey().mapValues(len).take(5)

In [ ]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda x : "ARG" in x[2])
equiposArgentinos.take(5)

In [ ]:
equiposOlimpicosRDD.count() # ESTE CUENTA TODOS LOS ARCHIVOS, POR LO QUE PUEDE LLEGAR A TARDAR

In [ ]:
equiposOlimpicosRDD.countApprox(1)  # SOLAMENTE CUENTA X MILISEGUNDOS

In [ ]:
#sc.stop()

In [ ]:
!ls curso-apache-spark-platzi/

In [ ]:
deportistaOlimpicoRDD = sc.textFile(path + "deportista.csv") \
                        .map(lambda l : l.split(","))
deportistaOlimpicoRDD2 = sc.textFile(path + "deportista2.csv") \
                        .map(lambda l : l.split(","))
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [ ]:
equiposOlimpicosRDD.top(2)

In [ ]:
deportistaOlimpicoRDD.top(5)

In [ ]:
deportistaOlimpicoRDD.map(lambda l : [l[-1],l[:-1]]) \
        .join(equiposOlimpicosRDD.map(lambda l : [l[0],l[2]])) \
        .takeSample(False,6,25)

In [ ]:
resultado = sc.textFile(path + "resultados.csv") \
            .map(lambda l : l.split(","))
resultado.take(5)

In [ ]:
resultadoGanador = resultado.filter(lambda l : 'NA' not in l[1])

In [ ]:
resultadoGanador.top(5)

In [ ]:
paises_medallas = deportistaOlimpicoRDD.map(lambda l : [l[-1],l[:-1]]) \
        .join(equiposOlimpicosRDD.map(lambda l : [l[0],l[2]])) \
        .map(lambda l : [l[1][0][0],[l[0],[l[1][0][1:],l[1][1]]]]) \
        .join(resultadoGanador.map(lambda l : [l[2],l[:2]])) \
        #.takeSample(False,6,25)

In [ ]:
#sc.stop()

In [ ]:
valores_medallas = {"Gold": 7,
                   "Silver" : 5,
                   "Bronze" : 4}

In [ ]:
paises_medallas.take(5)

In [ ]:
paises_medallas = paises_medallas.map(lambda l : [l[1][0][-1][-1],valores_medallas[l[1][1][1]]])

In [ ]:
from operator import add
paises_medallas.reduceByKey((add)) \
.sortBy(lambda x : x[1],ascending = False).take(10)

In [ ]:
#sc.stop()